# spannerlog Magic

In [ ]:
#| default_exp magic

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from typing import Optional
import logging
logger = logging.getLogger(__name__)

from pathlib import Path
import pandas as pd
from IPython.core.magic import (Magics, magics_class, line_cell_magic)
from singleton_decorator import singleton
from spannerlib.utils import assert_df_equals
from spannerlib.session import Session

In [ ]:
#| export
@singleton
class _MagicSession():
    def __init__(self):
        self.session = Session()

def get_magic_session():
    """Returns the session used by the magic system

    Returns:
        Session
    """    
    return _MagicSession().session

def set_magic_session(session: Session):
    """Changes the session used by the magic system to the one provided

    Args:
        session (Session): the session to use in the magic system
    """
    _MagicSession().session = session


In [ ]:
sess_1 = Session()
sess_1

<spannerlib.session.Session>

In [ ]:
assert get_magic_session()==get_magic_session()
assert get_magic_session()!=sess_1
set_magic_session(sess_1)
assert get_magic_session()==sess_1

In [ ]:
x= Path('test.txt')
x.write_text?

Signature: x.write_text(data, encoding=None, errors=None, newline=None)
Docstring: Open the file in text mode, write to it, and close the file.
File:      ~/miniconda3/envs/span/lib/python3.11/pathlib.py
Type:      method

In [ ]:
#| export
import argparse
import shlex
magic_args_parser = parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output",type=str, help="write code to file")
parser.add_argument("-a", "--append",type=str, help="append code to file")

def parse_options(line):
    return magic_args_parser.parse_args(shlex.split(line))

@magics_class
class spannerlogMagic(Magics):
    @line_cell_magic
    def spannerlog(self, line: str, cell: str) -> None:
        # import locally to prevent circular import issues
        magic_session = get_magic_session()

        args = parse_options(line)

        code = cell if cell else line
        _ = magic_session.export(code,display_results=True)

        if args.output:
            out_file = Path(args.output)
            out_file.write_text(code + '\n')
            
        if args.append:
            out_file = Path(args.append)
            if not code in out_file.read_text():
                with open(out_file, "a") as f:
                    f.write(code + '\n')

def load_ipython_extension(ipython):
    ipython.register_magics(spannerlogMagic)

In [ ]:
try:
    load_ipython_extension(get_ipython())
    logger.warning("spannerlog magic loaded")
except (AttributeError, ImportError):
    pass

spannerlog magic loaded


In [ ]:
%%spannerlog
new A(int)
A(1)
?A(X)

'?A(X)'

,X
0,1


In [ ]:
session = get_magic_session()

assert_df_equals(session.export("?A(X)"),pd.DataFrame({'X':[1]}))

,X
0,1


In [ ]:
%%spannerlog -o text.slog
new A(int)

In [ ]:
%%spannerlog -a text.slog
new B(int)

In [ ]:
%%spannerlog -a text.slog
new B(int)

In [ ]:
assert Path('text.slog').read_text() == 'new A(int)\n\nnew B(int)\n\n'
Path('text.slog').unlink()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     